<a href="https://colab.research.google.com/github/Marukos/Decision-Trees/blob/main/DecisionTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that.

Fill in your **NAME** and **AEM** below:

In [ ]:
NAME = "Markos Koletsas"
AEM = "3557"

---

# Assignment 2 - Decision Trees #

Welcome to your second assignment. This exercise gives you an introduction to [scikit-learn](https://scikit-learn.org/stable/). A simple but efficient machine learning library in Python. It also gives you a wide understanding on how decision trees work.

After this assignment you will:
- Be able to use the scikit-learn library and train your own model from scratch.
- Be able to train and understand decision trees.

In [ ]:
# Always run this cell
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# USE THIS RANDOM VARIABLE TO PRODUCE THE SAME RESULTS
RANDOM_VARIABLE = 42

## 1. Scikit-Learn and Decision Trees ##

You are going to use the scikit-learn library to train a model for detecting breast cancer using the [Breast cancer wisconsin (diagnostic) dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) (+ [Additional information](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset)) by training a model using [decision trees](https://scikit-learn.org/stable/modules/tree.html).

**1.1** Load the breast cancer dataset using the scikit learn library and split the dataset into train and test set using the appropriate function. Use 33% of the dataset as the test set. Define as X the attributes and as y the target values. Do not forget to set the random_state parameter as the *RANDOM_VARIABLE* defined above. Use this variable for all the random_state parameters in this assignment.

In [ ]:
from pandas.core.common import random_state
# BEGIN CODE HERE
X, y = load_breast_cancer().data, load_breast_cancer().target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_VARIABLE, test_size=0.33)

#END CODE HERE

In [ ]:
print("Size of train set:{}".format(len(y_train)))
print("Size of test set:{}".format(len(y_test)))
print("Unique classes:{}".format(len(set(y_test))))

Size of train set:381
Size of test set:188
Unique classes:2


**Expected output**:  

```
Size of train set:381  
Size of test set:188  
Unique classes:2
```



**1.2** Train two DecisionTree classifiers and report the F1 score. Use the information gain for the one classifier and the Gini impurity for the other

In [ ]:
# BEGIN CODE HERE
classifier_gini = DecisionTreeClassifier(random_state=RANDOM_VARIABLE, criterion='gini')
classifier_igain = DecisionTreeClassifier(random_state=RANDOM_VARIABLE, criterion='entropy')

classifier_gini.fit(X_train, y_train)
classifier_igain.fit(X_train, y_train)

prediction_gini = classifier_gini.predict(X_test)
prediction_igain = classifier_igain.predict(X_test)

f_measure_gini = f1_score(prediction_gini,y_test)
f_measure_igain = f1_score(prediction_igain,y_test)

#END CODE HERE

In [ ]:
print("F-Measure Gini: {}".format(f_measure_gini))
print("F-Measure Information Gain: {}".format(f_measure_igain))

F-Measure Gini: 0.9372384937238494
F-Measure Information Gain: 0.9596774193548386


**Expected output**:  

```
F-Measure Gini: 0.9372384937238494
F-Measure Information Gain: 0.9596774193548386
```

**1.3** Find the maximum depth reached by the tree that used the Gini impurity. Train multiple classifiers by modifying the max_depth within the range from 1 to maximum depth and save the f1 scores to the corresponding list of the *fscores* dictionary (one list for training set and one for test set). Before appending the scores to the corresponding list, multiply them by 100, and round the values to 2 decimals.

In [ ]:
# BEGIN CODE HERE
depth = classifier_gini.get_depth()
fscores = {}
fscores['train'] = []
fscores['test'] = []

for i in range(0, depth):
    classifier_gini2 = DecisionTreeClassifier(random_state=RANDOM_VARIABLE, criterion='gini', max_depth=i+1)
    classifier_gini2.fit(X_train, y_train)
    prediction_gini_train = classifier_gini2.predict(X_train)
    prediction_gini_test = classifier_gini2.predict(X_test)
    f_measure_gini_train = f1_score(prediction_gini_train,y_train)
    f_measure_gini_test = f1_score(prediction_gini_test,y_test)
    fscores['train'].append(round(f_measure_gini_train*100,2))
    fscores['test'].append(round(f_measure_gini_test*100,2))
#END CODE HERE

In [ ]:
print("Fscores Train: {}".format(fscores['train']))
print("Fscores Test:  {}".format(fscores['test']))


Fscores Train: [94.24, 95.46, 97.65, 99.15, 99.37, 99.58, 100.0]
Fscores Test:  [91.14, 93.97, 96.64, 94.12, 95.4, 95.04, 93.72]


**Expected output**:  
```
Fscores Train: [94.24, 95.46, 97.65, 99.15, 99.37, 99.58, 100.0]
Fscores Test:  [91.14, 93.97, 96.64, 94.12, 95.4, 95.04, 93.72]
```

**1.4** Compare the results from the train set with the results from the test set. What do you notice? How are you going to choose the max_depth of your model?

Παρατηρούμε πως το score κατά την εκπαίδευση συνεχώς αυξάνεται, όσο αυξάνουμε και το βάθος με αποτέλεσμα να καταφέρνει να φτάσει μέχρι και το 100%. Παρόλα αυτά είμαστε βέβαιοι πριν καν μελετήσουμε τα αποτελέσματα κατά τον έλεγχο πως πρόκειται για μια περίπτωση υπερεκπαίδευσης.

Προχωρώντας και μελετώντας και τα αποτελέσματα κατά τον έλεγχο παρατηρούμε πως η υπόθεση μας ήταν σωστή. Μέχρι και το μέγιστο_βάθος=3 το ποσοστό επιτυχίας αυξάνεται, ωστόσο αυτό αποτελεί σημείο καμπής και ύστερα αρχίζει να μειώνεται σημειώνοντας σημαντική πτώση από 96.64% σε 93.72%.

Επομένως, η τιμή που θα επιλέγαμε θα ήταν μέγιστο_βάθος=3, καθώς μας ενδιαφέρει περισσότερο το ποσοστό επιτυχίας κατά τον έλεγχο (δηλαδή σε άγνωστα δεδομένα), παρά κατά την εκπαίδευση.

## 2.0 Pipelines ##

**2.1** In this part of the exercise you are going to build a pipeline from scratch for a classification problem. Load the **income.csv** file and train a DecisionTree model that will predict the *income* variable. This dataset is a modification of the original Adult Income dataset found [here](http://archive.ics.uci.edu/ml/datasets/Adult). Report the f1-score and accuracy score of the test set found in **income_test.csv**. Your pipeline should be able to handle missing values and categorical features (scikit-learn's decision trees do not handle categorical values). You can preprocess the dataset as you like in order to achieve higher scores.  

In [ ]:
# BEGIN CODE HERE
train_set = pd.read_csv('income.csv').drop('income', axis=1)
y_train = pd.read_csv('income.csv')['income']
y_train = pd.DataFrame(columns=['income'], data=y_train)
# any other code you need
encoder = OrdinalEncoder()
y_train = encoder.fit_transform(y_train)

test_set = pd.read_csv('income_test.csv').drop('income', axis=1)
y_test = pd.read_csv('income_test.csv')['income'].values
y_test = pd.DataFrame(columns=['income'], data= y_test)
# any other code you need
y_test = encoder.transform(y_test)
# End CODE HERE

**2.2** Create and test your pipeline

In [ ]:
#Your pipeline
numeric_features = ["hours-per-week", "age","education_num","capital-gain","capital-loss"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean"))]
)

nan_numeric_values = ["fnlwgt"]
nan_numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent"))]
)

categorical_features = ['marital-status', 'education', 'relationship', 'sex', 'race', 'workclass', 'occupation']
categorical_transformer = Pipeline(steps=[
           ("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder())
])

ordinal_features = []
ordinal_transformer = Pipeline (
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OrdinalEncoder())]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("num_fw", nan_numeric_transformer, nan_numeric_values),
        ("one_hot_cat", categorical_transformer, categorical_features),
        ("ordinal_cat", ordinal_transformer,ordinal_features)
    ]
)


clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", DecisionTreeClassifier(random_state=RANDOM_VARIABLE))]
)

clf.fit(train_set, y_train)
y_predict =  clf.predict(test_set)

In [ ]:
print("Model score Accuracy: %.3f" % accuracy_score(y_test, y_predict))
print("Model score F1 Weighted: %.3f" % f1_score(y_test, y_predict,average='weighted'))

Model score Accuracy: 0.806
Model score F1 Weighted: 0.807


**2.3** Perform a gooood grid search to find the best parameters for your pipeline

In [ ]:
param_grid = {
    "preprocessor__num__imputer__strategy": ["mean", "median"],
    "classifier__max_depth": [10, 19, 35],
    "classifier__criterion": ["gini","entropy"],
    "classifier__max_features": [0.25, 0.5, 0.75, None],
    "classifier__min_samples_leaf": [1, 3, 7],
    "classifier__max_leaf_nodes": [70, 80, 90, 105]
}

grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(train_set, y_train)
y_predict =  grid_search.predict(test_set)

print("Best params:")
print(grid_search.best_params_)

Best params:
{'classifier__criterion': 'gini', 'classifier__max_depth': 35, 'classifier__max_features': None, 'classifier__max_leaf_nodes': 90, 'classifier__min_samples_leaf': 7, 'preprocessor__num__imputer__strategy': 'mean'}


In [ ]:
print("Model score Accuracy: %.3f" % accuracy_score(y_test,y_predict))
print("Model score F1 Weighted: %.3f" % f1_score(y_test,y_predict,average='weighted'))

Model score Accuracy: 0.857
Model score F1 Weighted: 0.853


**2.4** Describe the process you followed to achieve the results above. Your description should include, but is not limited to the following
- How do you handle missing values and why
- How do you handle categorical variables and why
- Any further preprocessing steps
- How do you evaluate your model and how did you choose its parameters
- Report any additional results and comments on your approach.

You should achieve at least 85% accuracy score and 84% f1 score.

Αφού φορτώσουμε το dataset, η πρώτη κίνηση μας είναι να μετατρέψουμε τους στόχους μας (δηλαδή το y_train και το y_test) σε συνεχείς τιμές, οπότε πλέον από "<=50Κ" και ">50Κ" οι κλάσεις μας θα ορίζονται από 0 και 1, αυτό το πετυχαίνουμε χρησιμοποιώντας ένας Ordinal Encoder. Το πρώτο εμπόδιο μας είναι τα missing values.

Στο συγκεκριμένο dataset υπάρχουν μόνο κατηγορηματικές τιμές που λείπουν, παρόλα αυτά το αγνοήσαμε αυτό και για λόγους εξάσκησης είδαμε πως θα αντιμετωπίζαμε και τις αριθμητικές τιμές που λείπουν.
*   Για τις αριθμητικές μεταβλητές λάβαμε δύο χωριστές περιπτώσεις, μια ήταν αυτή στην οποία θέλαμε να δώσουμε σε όλα τα missing values τη μέση τιμή της συγκεκριμένης μεταβλητής και στην άλλη κατηγορία όσες θέλαμε να έχουμε τη πιο συχνά χρησιμοποιούμενη τιμή. Στην δεύτερη περίπτωση άνηκε μόνο η μεταβλητή fnlwgt, καθώς αυτός ο αριθμός προσδίδει χαρακτηριστικά στο συγκεκριμένο άτομο και δεν έχει αριθμητική αξία που μας νοιάζει το μέγεθος της. Όλες οι υπόλοιπες αριθμητικές μεταβλητές αντιμετωπίστηκαν με τη στρατηγική του mean.
*   Για τις κατηγορηματικές μεταβλητές χρησιμοποιήσαμε σε όλες τη στρατηγική του most frequent, γιατί διαφορετικά η μόνη άλλη στρατηγική που μπορεί να χρησιμοποιηθεί είναι αυτή που ορίζουμε σε όλα τα missing values μιας μεταβλητής την ίδια τιμή. Ωστόσο, αυτή η στρατηγική θα μπορούσε να χρησιμοποιηθεί μόνο από έναν ειδικό του πεδίου και μπορεί πάλι να μην έδινε ικανοποιητικα αποτελέσματα.

Αφού πλέον δεν υπάρχουν κενές τιμές έχει σειρά να μετατρέψουμε όλες τις κατηγορηματικές μεταβλητές σε αριθμητικές, ώστε να είναι συμβατές με το Decision Tree Classifier που μας προμηθεύει η βιβλιοθήκη scikit learn. Πάλι πρέπει να χωρίσουμε τις κατηγορηματικές μας μεταβλητές σε δύο ομάδες, αυτές στις οποίες θα χρησιμοποιήσουμε Ordinal Encoder και αυτές στις οποίες θα χρησιμοποιήσουμε One Hot Encoder. Αυτή τη φορά όμως αντί να διαλέξουμε πως θα γίνει ο χωρισμός αυτός βάσει της λογικής χρησιμοποιήσαμε brute force και ελέγξαμε την απόδοση του μοντέλου για κάθε δυνατό συνδυασμό (όχι μεταθέσεις λόγω της αυξημένης χρονικής πολυπλοκότητας). Αυτή η διαδικασία έγινε δύο φορές μία χωρίς να δώσουμε παραμέτρους στο μοντέλο μας και μία με τις παραμέτρους που προέκυψαν από το grid search. Ο κώδικας για τη συγκεκριμένη διαδικασία υπάρχει στο τέλος του notebook.

Ξεκινήσαμε επιλέγοντας για ποιες παραμέτρους θέλουμε να βρούμε τις βέλτιστες τιμές. Αφού διαλέξαμε τις παραμέτρους σύμφωνα με το ποιες θεωρήσαμε οι ίδιοι πως έχουν τη μεγαλύτερη βαρύτητα, έπρεπε να διαλέξαμε ορισμένες τιμές για τις οποίες θέλαμε να δούμε την απόδοση του μοντέλου μας. Τη πρώτη φορά κινηθήκαμε χωρίζοντας, όπου ήταν εφικτό, το διάστημα από το 1 μέχρι τη τιμή που είχε δώσει το αρχικό μας μοντέλο στη συγκεκριμένη παράμετρο. Ύστερα, βλέποντας τα αποτελέσματα και τις τιμές που επιλέγονταν για κάθε παράμετρο επαναλαμβάναμε τη διαδικασία για τιμές πλέον πιο κοντά σε αυτή που πετυχέναμε τη μέγιστη απόδοση.

Αφού ρυθμίσουμε τις παραμέτρους του Pipeline εκτελούμε ξανά το brute force αλγόριθμο για να αλλάξουμε ποιες κατηγορηματικές μεταβλητές θα γίνουν transform με ordinal encoder και ποιες με one hot encoder. Παραδόξως, μας εμφανίζει πως τα καλύτερα αποτελέσματα τα λαμβανουμε αν χρησιμοποιήσουμε μόνο τον One hot encoder.

Έτσι, καταφέραμε να αυξήσουμε την απόδοση μας ελάχιστα και να τη φέρουμε στο 85.7% accuracy score και 85.2% f1 score.




## 3.0 Common Issues ##

**3.0** Run the following code to define a DecisionTreeModel and load the **income** dataset only with the numerical variables. Then, answer the following questions.

In [ ]:
# Load Data
columns = ['age','fnlwgt','education_num','hours-per-week',"capital-loss","capital-gain","income"]
data = pd.read_csv('income.csv',usecols=columns)
data_test = pd.read_csv('income_test.csv',usecols=columns)
# Convert target variable to 0 and 1
data["income"] = data["income"].map({ "<=50K": 0, ">50K": 1 })
data_test["income"] = data_test["income"].map({ "<=50K": 0, ">50K": 1 })
# Create X and y
X_train = data.drop(["income"],axis=1)
y_train = data['income'].values
X_test = data_test.drop(["income"],axis=1)
y_test = data_test['income'].values
# Classifier
classifier = DecisionTreeClassifier(min_samples_leaf=4)
classifier.fit(X_train,y_train)
y_predict = classifier.predict(X_test)
accuracy_score = accuracy_score(y_test,y_predict)
print("Model score accuracy: %.3f" % accuracy_score)

Model score accuracy: 0.791


**3.1** Evaluate the classifier using at least three evaluation metrics except accuracy_score and f1 (weighted).

In [ ]:
from sklearn.metrics import balanced_accuracy_score, average_precision_score, f1_score, accuracy_score
# y_predict = classifier.predict(X_test)

# BEGIN CODE HERE
metric1 = balanced_accuracy_score(y_test,y_predict)
metric2 = average_precision_score(y_test,y_predict)
metric3 = f1_score(y_test,y_predict,average = "binary")
# END CODE HERE

In [ ]:
print("Model score Metric 1: %.3f" % metric1)
print("Model score Metric 2: %.3f" % metric2)
print("Model score Metric 3: %.3f" % metric3)

Model score Metric 1: 0.688
Model score Metric 2: 0.414
Model score Metric 3: 0.533


**3.2** Do you notice any problems with the classifier? If so, what can you do to change this.

Το κυριότερο πρόβλημα του κατηγοριοποιητή μας είναι η απουσία ρύθμισης των υπερπαραμέτρων του, επομένως έχοντας της default υπερπαραμέτρους της κλάσης του αυτές πολύ πιθανόν να μην είναι οι κατάλληλες για τα δεδομένα μας. Ένας τρόπος για να επιλυθεί θα ήταν να εφαρμόσουμε grid search.

Επίσης, ένα άλλο πρόβλημα που ενδέχεται να υπάρχει είναι πως τα δεδομένα μπορεί να μην είναι ομοιόμορφα χωρισμένα, δηλαδή για τη μεταβλητή income που έχει τιμές 0 και 1, το train set μας να αποτελείται από 80% 0, ενώ το test set μας από 20% 0. Με αποτέλεσμα να μη μπορεί να γίνει το κατάλληλο fitting. Για να το λύσουμε αυτό θα κάνουμε stratification.



**3.3** Implement your solution using the cells below. Report your results and the process you followed. You are reccommended to use stratification and grid search. You should only have to increase a little bit the metrics you calculated above, and also reach an accuracy score higher than 82%!

In [ ]:
# BEGIN CODE HERE
final_score = "0.836"

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

columns = ['age','fnlwgt','education_num','hours-per-week',"capital-loss","capital-gain","income"]
data = pd.read_csv('income.csv', usecols=columns)
data_test = pd.read_csv('income_test.csv',usecols=columns)
data = pd.concat([data, data_test])
data["income"] = data["income"].map({ "<=50K": 0, ">50K": 1 })

sss = StratifiedShuffleSplit(n_splits = 10, test_size = data_test.shape[0]/data.shape[0] ,
                             random_state = RANDOM_VARIABLE)
X = data.drop(["income"],axis=1).values
y = data['income'].values
best = [0.0,[],[],[],[]]
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier = DecisionTreeClassifier(min_samples_leaf=4)
    classifier.fit(X_train,y_train)
    y_predict = classifier.predict(X_test)
    acc_score = accuracy_score(y_test,y_predict)
    if acc_score > best[0]:
        best[0] = acc_score
        best[1] = X_train
        best[2] = y_train
        best[3] = X_test
        best[4] = y_test

X_train, y_train, X_test, y_test = best[1:]

param_grid = {
    "max_depth": [1, 12, 25, 50],
    "criterion": ["gini","entropy"],
    "max_features": [0.25, 0.5, 0.75, None],
    "min_samples_leaf": [1, 5, 10],
    "max_leaf_nodes": [30, 60, 90],
    "random_state": [RANDOM_VARIABLE]
}

grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train, y_train)
y_predict = grid_search.predict(X_test)

print("Best params:")
print(grid_search.best_params_)

classifier.set_params(**grid_search.best_params_)
best = [0.0,[],[],[],[]]
for train_index, test_index in sss.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  classifier.fit(X_train,y_train)
  y_predict = classifier.predict(X_test)
  acc_score = accuracy_score(y_test,y_predict)
  if acc_score > best[0]:
    best[0] = acc_score
    best[1] = X_train
    best[2] = y_train
    best[3] = X_test
    best[4] = y_test

X_train, y_train, X_test, y_test = best[1:]
classifier.fit(X_train,y_train)
y_predict = classifier.predict(X_test)

accuracy_score = accuracy_score(y_test,y_predict)
metric1 = balanced_accuracy_score(y_test,y_predict)
metric2 = average_precision_score(y_test,y_predict)
metric3 = f1_score(y_test,y_predict,average = "binary")

#END CODE HERE

Best params:
{'criterion': 'gini', 'max_depth': 12, 'max_features': None, 'max_leaf_nodes': 60, 'min_samples_leaf': 1, 'random_state': 42}


In [ ]:
print("Model score accuracy: %.3f" % accuracy_score)
print("Model score Metric 1: %.3f" % metric1)
print("Model score Metric 2: %.3f" % metric2)
print("Model score Metric 3: %.3f" % metric3)

Model score accuracy: 0.836
Model score Metric 1: 0.711
Model score Metric 2: 0.488
Model score Metric 3: 0.581


Ξεκινάμε φορτώνοντας τις συναρτήσεις που θα χρειαστούμε, καθώς και το μέρος του dataset που επιλέξαμε να χρησιμοποιήσουμε (δηλαδή μόνο τις αριθμητικές τιμές) και παράλληλα κάνουμε και την κατάλληλη προεπεξεργασία στη μεταβλητή income για να της δώσουμε τη μορφή που επιθυμούμε.

Ύστερα συνενώνουμε τα δύο μέρη του dataset (train-set και test-set) σε ένα ενιαίο για να εφαρμόσουμε στρωματοποίηση και να το χωρίσουμε κατάλληλα εκ νέου σε train-set και test-set διατηρώντας το μέγεθος που είχαν και προηγουμένως. Έτσι, θα υπάρχει τέτοιος διαχωρισμός, όπου θα έχουμε σχεδόν το ίδιο ποσοστό κάθε τιμής κάθε μεταβλητής μεταξύ test-set και train-set.

Θα δημιουργήσουμε δέκα διαφορετικούς τέτοιους διαχωρισμούς και θα μετρήσουμε το accuracy για κάθε έναν από αυτούς με τον classifier που μας δίνεται, όποιος πετύχει το μεγαλύτερο score είναι και αυτός που θα κρατήσουμε.

Έπειτα, εκτελούμε grid search όπως είχε περιγραφεί και στο από πάνω ερώτημα. Διαλέγουμε ορισμένες παραμέτρους και διεξάγουμε πειράματα, ώσπου να βρούμε τις τιμές για ένα τοπικό μέγιστο στο οποίο θα συγκλίνει το μοντέλο μας. Αφού κάνουμε και αυτό το βήμα, εκτελούμε ξανά το προηγούμενο για να βρούμε το νέο διαχωρισμό του dataset μας που θα εξυπηρετεί το συγκεκριμένο μοντέλο μας, καθώς πλέον έχουμε αλλάξει τις παραμέτρους του.

Τέλος, υπολογίζουμε ξανά τις μετρικές μας για να δούμε κατά πόσο καταφέραμε να τις βελτιώσουμε.

Αφού δεν ορίζεται random_state στον classifier που μας δόθηκε το αριστερό μέρος μονάχα μπορεί να διαφέρει από run σε run, αλλά συνήθως θα είναι σε ακρίβεια τρίτου δεκαδικού ψηφίου. Τα τελικά μας αποτελέσματα ήταν τα εξής για το συγκεκριμένο run:

*   accuracy_score:          0.792 ➔ 0.836
*   average_precision_score: 0.415 ➔ 0.488
*   balanced_accuracy_score: 0.688 ➔ 0.711
*   f1_score (binary):       0.534 ➔ 0.581

Επομένως, παρατηρούμε αύξηση σε όλες τις προηγούμενες μετρικές μας.



# Πρόχειρο

In [ ]:
from itertools import combinations
from sklearn.metrics import accuracy_score
strings = {"occupation", "education", "marital-status","workclass","race", "sex", "relationship"}
max = [[{}, 0.0, 0.0,{}],[{}, 0.0, 0.0,{}],[{}, 0.0, 0.0,{}]]
for i in range (0,8):
  a = list(combinations(strings, 7-i))
  for j in a:
    categorical_features = list(j)
    for k in combinations(strings.difference(j), len(strings.difference(j))):
      ordinal_features = list(k)

      preprocessor = ColumnTransformer(
      transformers=[
          ("num", numeric_transformer, numeric_features),
          ("num_fw", nan_numeric_transformer, nan_numeric_values),
          ("one_hot_cat", categorical_transformer, categorical_features),
          ("ordinal_cat", ordinal_transformer,ordinal_features)
        ]
      )


      clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", DecisionTreeClassifier(random_state=RANDOM_VARIABLE,
                                                                                      max_depth = 24,
                                                                                      criterion ="gini",
                                                                                      max_features = None,
                                                                                      min_samples_leaf = 4,
                                                                                      max_leaf_nodes = 95))]
      )
      clf.fit(train_set, y_train)
      y_predict =  clf.predict(test_set)
      if accuracy_score(y_test, y_predict) > max[0][1]:
        max[0][0] = categorical_features
        max[0][1] = accuracy_score(y_test, y_predict)
        max[0][2] = f1_score(y_test, y_predict,average='weighted')
        max[0][3] = ordinal_features
      if f1_score(y_test, y_predict,average='weighted') > max[1][2]:
        max[1][0] = categorical_features
        max[1][1] = accuracy_score(y_test, y_predict)
        max[1][2] = f1_score(y_test, y_predict,average='weighted')
        max[1][3] = ordinal_features
      if accuracy_score(y_test, y_predict) > max[2][1] and f1_score(y_test, y_predict,average='weighted') > max[2][2]:
          max[2][0] = categorical_features
          max[2][1] = accuracy_score(y_test, y_predict)
          max[2][2] = f1_score(y_test, y_predict,average='weighted')
          max[2][3] = ordinal_features

print(max)

[[['education', 'workclass', 'sex', 'race', 'marital-status'], 0.7544237675481554, 0.6540372762486302, ['relationship', 'occupation']], [['workclass', 'race', 'marital-status', 'occupation'], 0.7535749265426053, 0.6566236312895299, ['education', 'sex', 'relationship']], [['education', 'workclass', 'sex', 'race', 'marital-status', 'occupation'], 0.7539014038524322, 0.6556420917541126, ['relationship']]]
